# Performance Python

Here we will talk about a few ways to make code run faster. There are a number of ways to do this, but we will talk about three of the most standard approaches.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### Numba -- I want my code to run faster.

[Numba](http://numba.pydata.org/) is a fairly new package that is easier than previous approaches, such as [weave](https://docs.scipy.org/doc/scipy-0.18.1/reference/tutorial/weave.html) and [cython](http://cython.org/).

Numba uses [decorators](http://simeonfranklin.com/blog/2012/jul/1/python-decorators-in-12-steps/) to identify python functions that sould be pre-compiled into C code to run faster. Decorators are basically just wrapper functions -- that is it runs the decorated function inside another function (identified by @decorator_name in front of the function) so that the inputs and outputs might be modified. But for now, you can just think of decorators as flags that identify which functions you want to run fast.

Numba has a _just-in-time_ compiler that will only compile the function before it is called the first time. The associated decorator is `numba.jit`. 

In [ ]:
from numba import jit

@jit
def julia_fast(x, y, c, zabs_max=10, nit_max = 1000):  
    res = np.zeros((len(y), len(x)))
    for i, ix in enumerate(x):
        for j, iy in enumerate(y):
            z = complex(ix, iy)
            nit = 0
            while abs(z) <= zabs_max and nit < nit_max:
                z = z**2 + c
                nit += 1

            res[i,j] = nit / nit_max

    return res
                   
c = complex(-0.5, 0.65)

x = np.linspace(-1.5, 1.5, 1001)
y = np.linspace(-1.5, 1.5, 1001)

In [ ]:
%timeit j = julia_fast(x, y, c)

Let's compare this to the original function, without the speed advantages of numba

In [ ]:
def julia_slow(x, y, c, zabs_max=10, nit_max = 1000):  
    res = np.zeros((len(y), len(x)))
    for i, ix in enumerate(x):
        for j, iy in enumerate(y):
            z = complex(ix, iy)
            nit = 0
            while abs(z) <= zabs_max and nit < nit_max:
                z = z**2 + c
                nit += 1

            res[i,j] = nit / nit_max

    return res

In [ ]:
%timeit j = julia_slow(x, y, c)

### Dask -- my code is too big to hold in memory

[Dask](http://dask.pydata.org/en/latest/) is a tool that allows for automatic parallelization of tasks.

In [ ]:
import dask.array as da

In [ ]:
# Dask is not particularly fast...

x = np.arange(1000000).reshape(1000, 1000)
y = da.from_array(x, chunks=(100))   # split the dataset into 100 chunks,  
%timeit y.mean().compute()           # to do computations on each chunk individually. 

%timeit x.mean()                     # ... but it's not faster if it fits in memory

In [ ]:
y.dask

Dask can run complex calculations on these split arrays by using [graphs](http://dask.pydata.org/en/latest/graphs.html). Refer to the documentation for more information.

### f2py -- I already have some fast FORTRAN code.

Use [f2py](https://docs.scipy.org/doc/numpy-dev/f2py/) primariy if you have some existing fortran code that you want to link with Python. Other libraries let you link C code in a similar way. 

In [ ]:
!cat julia.f

In [ ]:
!f2py -c -m julia julia.f

In [ ]:
!ls julia*

In [ ]:
import julia

In [ ]:
c = complex(-0.5, 0.65)

x = np.linspace(-1.5, 1.5, 1001)
y = np.linspace(-1.5, 1.5, 1001)

j = julia.julia(x, y, c, 5, 1000)

In [ ]:
j